In [1]:
import torch
from PIL import Image, ImageDraw, ImageFont
import os
import numpy as np
import sys
import glob

# --- 로컬 Pororo 모듈 경로 설정 ---
# 이 노트북 파일(predict.ipynb)은 customOCR 폴더에 있으므로,
# pororo_main 폴더는 같은 레벨에 있다고 가정합니다.
PORORO_PATH = os.path.abspath(os.path.join(os.path.dirname('.'), '..', 'pororo_easyocr_main'))
if PORORO_PATH not in sys.path:
    sys.path.append(PORORO_PATH)
    print(f"Added to sys.path: {PORORO_PATH}")

try:
    from main import EasyPororoOcr, BaseOcr
    import cv2
    from abc import ABC, abstractmethod
    from pororo import Pororo
    from pororo.pororo import SUPPORTED_TASKS
    from utils.image_util import plt_imshow, put_text
    from utils.image_convert import convert_coord, crop
    from utils.pre_processing import load_with_filter, roi_filter
    from easyocr import Reader
except ImportError:
    print(f"오류: '{PORORO_PATH}' 경로에서 Pororo 모듈을 찾을 수 없습니다.")
    print("customOCR 폴더와 같은 위치에 'pororo_main' 폴더가 있는지 확인해주세요.")
    # In a notebook, we might not want to exit, just raise the error.
    raise

from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification
# config.py 파일이 src 폴더 안에 있으므로 경로를 추가해줍니다.
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('.'), 'src')))
from config import OUTPUT_DIR, id2label, DEVICE

print("\n필요한 모듈을 모두 로드했습니다.")

Added to sys.path: c:\code\pororo_easyocr_main

필요한 모듈을 모두 로드했습니다.


In [ ]:
import re
import torch.nn.functional as F

class DocumentPredictor:
    """
    학습된 LayoutLMv3 모델을 사용하여 문서 정보를 추출하는 클래스.
    (OCR 엔진을 EasyPororoOcr로 교체하여 정확도 향상)
    """
    def __init__(self, model_path, confidence_threshold=0.85):
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"모델 경로를 찾을 수 없습니다: {model_path}. train.py를 먼저 실행하여 모델을 학습시키세요.")
        
        print("모델과 프로세서를 로딩합니다...")
        self.processor = LayoutLMv3Processor.from_pretrained(model_path)
        self.model = LayoutLMv3ForTokenClassification.from_pretrained(model_path)
        self.model.to(DEVICE)
        self.model.eval()
        
        # ★★★ 변경점 1: __init__에서 ocr_reader 초기화 코드 제거 ★★★
        # self.ocr_reader = EasyPororoOcr(gpu=torch.cuda.is_available())
        self.confidence_threshold = confidence_threshold
        print(f"설정된 신뢰도 임계값: {self.confidence_threshold}")

    def _preprocess_image(self, image, top_crop_ratio=0.12, bottom_crop_ratio=0.08):
        """이미지의 상단/하단을 잘라내어 불필요한 UI 요소를 제거합니다."""
        width, height = image.size
        top_crop = int(height * top_crop_ratio)
        bottom_crop = int(height * (1 - bottom_crop_ratio))
        
        cropped_image = image.crop((0, top_crop, width, bottom_crop))
        return cropped_image, top_crop

    def _split_boxes_aggressively(self, ocr_words, ocr_boxes):
        """OCR 결과를 공백 기준으로 분할하되, 특정 패턴은 예외 처리합니다."""
        new_words, new_boxes = [], []
        exception_pattern = re.compile(r'(\d{1,2}월\s\d{1,2}일|\d{1,4}\.\d{1,2}\.\d{1,2}|\d{1,2}:\d{1,2})')
        for word, box in zip(ocr_words, ocr_boxes):
            if exception_pattern.fullmatch(word):
                new_words.append(word)
                new_boxes.append(box)
                continue
            parts = word.split()
            if len(parts) > 1:
                new_words.extend(parts)
                new_boxes.extend([box] * len(parts))
            else:
                new_words.append(word)
                new_boxes.append(box)
        return new_words, new_boxes

    def predict(self, image_path):
        """단일 이미지에 대해 OCR 및 정보 추출을 수행합니다."""
        # ★★★ 변경점 2: predict 함수가 호출될 때마다 OCR 엔진을 새로 생성 ★★★
        # 이렇게 하면 이미지마다 독립적인 OCR 상태를 유지하여 충돌을 방지합니다.
        print("OCR 엔진(EasyPororoOcr)을 초기화합니다...")
        ocr_reader = EasyPororoOcr(gpu=torch.cuda.is_available())

        print(f"\n'{os.path.basename(image_path)}'에서 텍스트를 추출합니다...")
        original_image = Image.open(image_path).convert("RGB")
        
        cropped_image, y_offset = self._preprocess_image(original_image.copy())
        
        # PIL 이미지를 OpenCV가 요구하는 정확한 데이터 타입으로 변환
        cropped_image_np = np.array(cropped_image)
        cropped_image_np = cv2.cvtColor(cropped_image_np, cv2.COLOR_RGB2BGR)
        if cropped_image_np.dtype != np.uint8:
            cropped_image_np = cropped_image_np.astype(np.uint8)

        # EasyPororoOcr 실행 및 결과 파싱
        ocr_reader.run_ocr(cropped_image_np, debug=False)
        ocr_results = ocr_reader.get_ocr_result()

        if not ocr_results:
            print("이미지에서 텍스트를 찾을 수 없습니다.")
            return [], original_image

        # EasyPororoOcr의 출력 형식에 맞게 단어와 박스 리스트를 생성
        raw_words_initial = [res[1] for res in ocr_results]
        raw_boxes_initial = []
        for res in ocr_results:
            box = res[0]
            x_coords = [p[0] for p in box]
            y_coords = [p[1] for p in box]
            # y_offset을 더해 원본 이미지 좌표로 변환
            raw_boxes_initial.append([min(x_coords), min(y_coords) + y_offset, max(x_coords), max(y_coords) + y_offset])

        words, raw_boxes = self._split_boxes_aggressively(raw_words_initial, raw_boxes_initial)

        width, height = original_image.size
        boxes = [[int(1000 * b[0] / width), int(1000 * b[1] / height), int(1000 * b[2] / width), int(1000 * b[3] / height)] for b in raw_boxes]

        encoding = self.processor(
            original_image, words, boxes=boxes, return_tensors="pt",
            padding="max_length", truncation=True,
        ).to(DEVICE)

        with torch.no_grad():
            outputs = self.model(**encoding)
        
        probabilities = F.softmax(outputs.logits, dim=-1)
        predictions = torch.argmax(probabilities, dim=-1)
        max_probs = torch.max(probabilities, dim=-1).values
        
        results = self._postprocess(encoding, predictions.squeeze().tolist(), max_probs.squeeze().tolist(), words, raw_boxes)
        
        return results, original_image

    def _postprocess(self, encoding, predictions, probabilities, words, raw_boxes):
        """
        예측 결과를 병합하고, 텍스트 내용과 숫자 패턴을 함께 고려하여 라벨을 재분류하는 후처리 함수.
        (규칙 우선순위 및 정규식 강화)
        """
        word_ids = encoding.word_ids(0)
        
        # 1. 신뢰도 기반 초기 필터링
        initial_preds = []
        previous_word_idx = None
        for idx, (pred_id, prob) in enumerate(zip(predictions, probabilities)):
            word_idx = word_ids[idx]
            if word_idx is None or word_idx == previous_word_idx:
                continue
            
            label = id2label[pred_id]
            if label != "O" and prob >= self.confidence_threshold:
                initial_preds.append({
                    "text": words[word_idx], "label": label, "box": raw_boxes[word_idx],
                    "confidence": prob, "word_idx": word_idx
                })
            previous_word_idx = word_idx

        if not initial_preds:
            return []

        # 2. 연속된 같은 라벨 병합
        merged_preds = []
        if initial_preds:
            current_pred = initial_preds[0]
            for i in range(1, len(initial_preds)):
                next_pred = initial_preds[i]
                if next_pred['label'] == current_pred['label'] and next_pred['word_idx'] == current_pred['word_idx'] + 1:
                    current_pred['text'] += " " + next_pred['text']
                    current_pred['box'][2] = next_pred['box'][2]
                    current_pred['box'][3] = max(current_pred['box'][3], next_pred['box'][3])
                    current_pred['confidence'] = max(current_pred['confidence'], next_pred['confidence'])
                    current_pred['word_idx'] = next_pred['word_idx']
                else:
                    merged_preds.append(current_pred)
                    current_pred = next_pred
            merged_preds.append(current_pred)

        # 3. ★★★ 키워드와 정규식을 함께 고려한 최종 재분류 ★★★
        final_preds = []
        for pred in merged_preds:
            text = pred['text']
            has_digits = any(char.isdigit() for char in text)

            if has_digits:
                # 금액 패턴(숫자, 쉼표, '원'으로 끝남)과 일치하는지 확인
                is_amount_pattern = re.search(r'[\d,]+원?$', text.strip())
                
                # 더 구체적인 '입금/출금'을 먼저 확인
                if '입금' in text or '+' in text or (pred['label'] == 'AMOUNT_IN' and is_amount_pattern):
                    pred['label'] = 'AMOUNT_IN'
                elif '출금' in text or '-' in text or (pred['label'] == 'AMOUNT_OUT' and is_amount_pattern):
                    pred['label'] = 'AMOUNT_OUT'
                # 그 다음 '잔액'을 확인
                elif '잔액' in text:
                    pred['label'] = 'BALANCE'
            
            pred['confidence'] = f"{pred['confidence']:.2f}"
            final_preds.append(pred)
            
        return final_preds

print("DocumentPredictor 클래스가 'EasyPororoOcr' 엔진과 함께 업데이트되었습니다.")

DocumentPredictor 클래스가 'EasyPororoOcr' 엔진과 함께 업데이트되었습니다.


In [3]:
def draw_predictions(image, predictions, font_path=None):
    """예측 결과를 원본 이미지 위에 시각화합니다. (신뢰도 표시 및 색상 개선)"""
    draw = ImageDraw.Draw(image)
    
    try:
        font = ImageFont.truetype(font_path or "malgun.ttf", size=15)
    except IOError:
        print("경고: 'malgun.ttf' 폰트를 찾을 수 없습니다. 기본 폰트를 사용합니다.")
        font = ImageFont.load_default()

    # ★★★ 시인성 좋은 색상으로 변경 ★★★
    label_colors = {
        "DATE_HEADER": "#ff7f0e", # 주황
        "DATE": "#1f77b4",       # 파랑
        "TIME": "#d62728",       # 빨강
        "MERCHANT": "#2ca02c",   # 초록
        "MEMO": "#9467bd",       # 보라
        "AMOUNT_IN": "#8c564b",  # 갈색
        "AMOUNT_OUT": "#e377c2", # 핑크
        "BALANCE": "#7f7f7f",    # 회색
    }

    for pred in predictions:
        box = pred['box']
        label = pred['label']
        color = label_colors.get(label, "#bcbd22") # 기본값: 올리브색
        
        draw.rectangle(box, outline=color, width=3)
        
        # ★★★ 라벨 텍스트에 신뢰도 점수 추가 ★★★
        label_text = f"{label} ({pred['confidence']})"
        
        text_bbox = draw.textbbox((box[0], box[1]), label_text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        
        label_bg_box = [box[0], box[1] - text_height - 6, box[0] + text_width + 8, box[1]]
        draw.rectangle(label_bg_box, fill=color)
        
        draw.text((box[0] + 4, box[1] - text_height - 4), label_text, fill="white", font=font)
        
    return image

print("draw_predictions 함수가 신뢰도 표시 기능과 새로운 색상 구성으로 업데이트되었습니다.")

draw_predictions 함수가 신뢰도 표시 기능과 새로운 색상 구성으로 업데이트되었습니다.


In [4]:
# --- 1. 설정 ---
# 테스트할 이미지가 있는 폴더
TEST_IMAGE_DIR = '../bank_statement_test'
TEST_IMAGE_DIR2 = '../bank_statement'
# 사용할 폰트 경로 (None으로 두면 시스템 기본 폰트 또는 'malgun.ttf' 시도)
FONT_PATH = None 

# --- 2. 예측기 초기화 ---
try:
    predictor = DocumentPredictor(model_path=OUTPUT_DIR)
except Exception as e:
    print(f"예측기 초기화 실패: {e}")
    # Stop execution if predictor fails
    raise

# --- 3. 이미지 예측 및 결과 출력 ---
test_image_files = glob.glob(os.path.join(TEST_IMAGE_DIR, '*.png')) + \
                   glob.glob(os.path.join(TEST_IMAGE_DIR, '*.jpg')) + \
                    glob.glob(os.path.join(TEST_IMAGE_DIR2, '*.png')) + \
                    glob.glob(os.path.join(TEST_IMAGE_DIR2, '*.jpg'))

if not test_image_files:
    print(f"오류: '{TEST_IMAGE_DIR}' 폴더에서 테스트 이미지를 찾을 수 없습니다.")
else:
    for image_path in test_image_files:
        # 예측 수행
        predictions, image = predictor.predict(image_path)
        
        # 결과 시각화
        result_image = draw_predictions(image.copy(), predictions, font_path=FONT_PATH)
        
        # 결과 출력 (이미지 및 텍스트)
        print(f"\n--- [{os.path.basename(image_path)}] 정보 추출 결과 ---")
        display(result_image) # Jupyter Notebook 환경에서 이미지 바로 표시
        
        print("\n[텍스트 요약]")
        if predictions:
            # 보기 좋게 라벨별로 묶어서 출력
            summary = {}
            for p in predictions:
                label = p['label']
                if label not in summary:
                    summary[label] = []
                summary[label].append(p['text'])
            
            for label, texts in summary.items():
                print(f"- {label}: {', '.join(texts)}")
        else:
            print("추출된 정보가 없습니다.")
        print("-" * 50)

모델과 프로세서를 로딩합니다...
OCR 엔진(EasyPororoOcr)을 초기화합니다...
설정된 신뢰도 임계값: 0.85

'KakaoTalk_20250624_075429567.png'에서 텍스트를 추출합니다...


AttributeError: 'EasyPororoOcr' object has no attribute 'detector'